In [330]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [331]:
# read the data
df = pd.read_csv('https://raw.githubusercontent.com/nikjohn7/Disaster-Tweets-Kaggle/main/data/train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [332]:
df.shape

(7613, 5)

In [333]:
# how many disaster tweets are there?
df[df['keyword'] == 'disaster'].shape

(35, 5)

In [334]:
df[df['keyword'] == 'disaster']

,id,keyword,location,text,target
2781,3998,disaster,"Los Angeles, London, Kent",I forgot to bring chocolate with me. Major dis...,0
2782,4000,disaster,"USA, Haiti, Nepal",More Natural Disaster Research Urgent http://t...,1
2783,4003,disaster,Portoviejo-Manabi-Ecuador,I'm a disaster?? https://t.co/VCV73BUaCZ,0
2784,4005,disaster,NaN,@LovelyLikeLaura I can see why one of your fav...,0
2785,4006,disaster,NaN,å¬'Only the sea knows how many are dead' @MSF_...,1
2786,4008,disaster,chillin at ceder rapids,Beautiful disaster // Jon McLaughlin is such a...,0
2787,4010,disaster,NaN,@Tim_A_Roberts w/o giving up too much of Nana ...,1
2788,4011,disaster,"Manila, Philippines",Strengthening partnerships #AfterHaiyan http:/...,1
2789,4012,disaster,en el pais de los arrechos,beautiful disaster https://t.co/qm5Sz0fyU8,0
2790,4014,disaster,NaN,My first staining attempt was a disaster https...,0


In [342]:
# how many non-disaster tweets are there?
df[df['keyword'] != 'disaster'].shape
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [337]:
import nltk
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))
include_stopwords = {'disaster'}
stopwords |= include_stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jacksonlu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [344]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [354]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.2)
# y_train

In [355]:
# Create a pipeline that first transforms the text data into TF-IDF vectors, then applies SVM
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=list(stopwords))),
    ('clf', svm.SVC()),
])

# Train the classifier
text_clf.fit(X_train, y_train)

# Predict the test set results
y_pred = text_clf.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred, target_names=['0', '1']))

              precision    recall  f1-score   support

           0       0.78      0.90      0.84       862
           1       0.84      0.67      0.75       661

    accuracy                           0.80      1523
   macro avg       0.81      0.79      0.79      1523
weighted avg       0.81      0.80      0.80      1523

